In [6]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Đọc dữ liệu
df = pd.read_csv('data_full_final.csv')

# --- THÊM MỚI: Xóa dòng không có tiêu đề (title) ---
# Dòng này sẽ xóa tất cả các hàng mà cột 'title' bị NaN (rỗng)
df = df.dropna(subset=['title'])

# 1. Hàm làm sạch Target (price_sale) -> Chuyển sang số để tính toán
def clean_price(price):
    if isinstance(price, str):
        # Xóa dấu chấm, chữ 'đ', khoảng trắng
        clean_str = re.sub(r'[^\d]', '', price)
        if clean_str:
            return float(clean_str)
    return np.nan

# Áp dụng hàm làm sạch để tạo cột số liệu (dùng cho Model)
df['price_sale_clean'] = df['price_sale'].apply(clean_price)

# --- XỬ LÝ ĐIỀN GIÁ TRỊ THIẾU (Thay đổi theo yêu cầu) ---

# A. Xử lý cột SỐ (price_sale_clean) để chạy mô hình
# Điền 18.490.000 (dạng số) vào các ô tính toán bị rỗng
df['price_sale_clean'] = df['price_sale_clean'].fillna(18490000)

# B. Xử lý cột CHỮ GỐC (price_sale) để xuất file CSV hiển thị cho đẹp
# 1. Điền vào các ô NaN
df['price_sale'] = df['price_sale'].fillna('18.490.000đ')
# 2. Tìm những dòng chứa chữ "liên hệ" (không phân biệt hoa thường) và thay thế
# Tham số case=False để bắt cả "Liên hệ", "LIÊN HỆ", "liên hệ"...
df.loc[df['price_sale'].str.contains('liên hệ', case=False, na=False), 'price_sale'] = '18.490.000đ'

# -------------------------------------------------------

# Tạo df_model để xử lý tiếp
df_model = df.copy()

# Reset index để đảm bảo KFold indexing hoạt động đúng sau khi dropna
df_model = df_model.reset_index(drop=True)

# 2. Phân rã (Decompose) cột gpu_info
def parse_gpu(info):
    if not isinstance(info, str):
        return pd.Series({'gpu_brand': 'Unknown', 'gpu_vram': 0, 'gpu_class': 'Unknown'})
    
    info_lower = info.lower()
    
    # Tách Brand
    brand = 'Other'
    if 'nvidia' in info_lower or 'geforce' in info_lower:
        brand = 'NVIDIA'
    elif 'amd' in info_lower or 'radeon' in info_lower:
        brand = 'AMD'
    elif 'intel' in info_lower:
        brand = 'Intel'
    elif 'apple' in info_lower:
        brand = 'Apple'
        
    # Tách VRAM (ví dụ: 4GB, 6g...)
    vram = 0
    vram_match = re.search(r'(\d+)\s*(gb|g)', info_lower)
    if vram_match:
        vram = int(vram_match.group(1))
    
    # Phân loại Card rời (Discrete) vs Card tích hợp (Integrated)
    gpu_class = 'Integrated'
    if vram > 0:
        gpu_class = 'Discrete'
    elif any(x in info_lower for x in ['rtx', 'gtx', 'quadro', 'mx']):
        gpu_class = 'Discrete'
    elif 'rx' in info_lower and 'radeon' in info_lower: 
        gpu_class = 'Discrete'
        
    return pd.Series({'gpu_brand': brand, 'gpu_vram': vram, 'gpu_class': gpu_class})

# Áp dụng hàm parse
gpu_features = df_model['gpu_info'].apply(parse_gpu)
df_model = pd.concat([df_model, gpu_features], axis=1)

# 3. K-Fold Target Encoding (Out-of-Fold)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
target_col = 'price_sale_clean'
cols_to_encode = ['gpu_brand', 'gpu_class']

# Khởi tạo cột mới
for col in cols_to_encode:
    df_model[f'{col}_te'] = np.nan

# Vòng lặp K-Fold để tính Target Encoding
for train_idx, val_idx in kf.split(df_model):
    # Tách fold
    X_tr = df_model.iloc[train_idx]
    X_val = df_model.iloc[val_idx]
    
    for col in cols_to_encode:
        # Tính trung bình target trên tập train của fold này
        means = X_tr.groupby(col)[target_col].mean()
        # Map giá trị trung bình vào tập validation của fold này
        df_model.loc[val_idx, f'{col}_te'] = X_val[col].map(means)

# Xử lý Missing Values sinh ra do Encoding
global_mean = df_model[target_col].mean()
for col in cols_to_encode:
    df_model[f'{col}_te'].fillna(global_mean, inplace=True)

# 4. Huấn luyện mô hình
# Chọn features
feature_cols = ['gpu_vram', 'gpu_brand_te', 'gpu_class_te']
X = df_model[feature_cols].fillna(0) # Fill 0 cho VRAM nếu còn sót
y = df_model[target_col]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42, max_depth=10),
    "Random Forest": RandomForestRegressor(random_state=42, n_estimators=100)
}

print("Kết quả đánh giá mô hình (chỉ dùng thông tin GPU):")
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"{name} -> MAE: {mae:,.0f} | R2 Score: {r2:.4f}")

# 5. Lưu kết quả
output_filename = 'data_gpu_encoded_oof.csv'
# Lưu df_model để giữ lại các cột đã xử lý (_te, _clean)
# Cột 'price_sale' trong file này bây giờ đã được điền "18.490.000đ" ở những chỗ thiếu
df_model.to_csv(output_filename, index=False)
print(f"\nĐã lưu file đã xử lý vào: {output_filename}")
print("Ghi chú: Các dòng thiếu giá hoặc 'liên hệ' đã được điền là '18.490.000đ' trong cột price_sale.")

C:\Users\DELL\AppData\Local\Temp\ipykernel_21244\3353411719.py:113: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_model[f'{col}_te'].fillna(global_mean, inplace=True)


Kết quả đánh giá mô hình (chỉ dùng thông tin GPU):
Linear Regression -> MAE: 6,182,873 | R2 Score: 0.1081
Decision Tree -> MAE: 5,812,034 | R2 Score: 0.1861
Random Forest -> MAE: 5,653,653 | R2 Score: 0.2512

Đã lưu file đã xử lý vào: data_gpu_encoded_oof.csv
Ghi chú: Các dòng thiếu giá hoặc 'liên hệ' đã được điền là '18.490.000đ' trong cột price_sale.


In [7]:
import pandas as pd
df = pd.read_csv('data_gpu_encoded_oof.csv')

# Danh sách các cột cần giữ lại
columns_to_keep = [
    'title',
    'gpu_info', 
    'screen_res', 
    'screen_tech', 
    'battery', 
    'gpu_brand', 
    'gpu_vram', 
    'gpu_class', 
    'gpu_brand_te', 
    'gpu_class_te'
]

# Tạo dataframe mới chỉ với các cột đã chọn
df_subset = df[columns_to_keep]

# Lưu ra file CSV mới
df_subset.to_csv('data_gpu_subset.csv', index=False)

print("Đã tạo file data_gpu_subset.csv thành công!")

Đã tạo file data_gpu_subset.csv thành công!


In [8]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('data_gpu_subset.csv')

#  Xử lý screen_res (One Hot Encoding) ---
# Làm sạch nhẹ (bỏ khoảng trắng thừa) trước khi one-hot
df['screen_res'] = df['screen_res'].astype(str).str.strip()
screen_res_dummies = pd.get_dummies(df['screen_res'], prefix='res')

#  Xử lý screen_tech (Phân rã + Encoding) 
# Hàm trích xuất độ sáng (nits)
def extract_brightness(text):
    if pd.isna(text): return np.nan
    match = re.search(r'(\d+)\s*nits', str(text), re.IGNORECASE)
    if match:
        return float(match.group(1))
    return np.nan

# Hàm trích xuất độ phủ màu (sRGB, NTSC, DCI-P3)
def extract_gamut(text, type_name):
    if pd.isna(text): return 0 
    # Tìm mẫu "100% sRGB" hoặc "sRGB 100%"
    pattern = r'(\d+)%\s*' + re.escape(type_name) + r'|' + re.escape(type_name) + r'\s*(\d+)%'
    match = re.search(pattern, str(text), re.IGNORECASE)
    if match:
        return float(match.group(1) or match.group(2))
    return 0 

# Hàm kiểm tra màn hình chống chói
def check_anti_glare(text):
    if pd.isna(text): return 0
    keywords = ['chống chói', 'anti-glare', 'antiglare']
    text_lower = str(text).lower()
    if any(k in text_lower for k in keywords):
        return 1
    return 0

# Áp dụng các hàm trên
df['screen_brightness_nits'] = df['screen_tech'].apply(extract_brightness)
df['screen_srgb_pct'] = df['screen_tech'].apply(lambda x: extract_gamut(x, 'sRGB'))
df['screen_ntsc_pct'] = df['screen_tech'].apply(lambda x: extract_gamut(x, 'NTSC'))
df['screen_dcip3_pct'] = df['screen_tech'].apply(lambda x: extract_gamut(x, 'DCI-P3'))
df['screen_anti_glare'] = df['screen_tech'].apply(check_anti_glare)

# Điền giá trị thiếu cho độ sáng bằng median (trung vị)
df['screen_brightness_nits'] = df['screen_brightness_nits'].fillna(df['screen_brightness_nits'].median())

#  Xử lý battery (Phân rã + Encoding) 
def extract_battery_wh(text):
    if pd.isna(text): return np.nan
    match = re.search(r'(\d+)\s*(?:Wh|WHr|WHrs)', str(text), re.IGNORECASE)
    if match:
        return float(match.group(1))
    return np.nan

def extract_battery_cells(text):
    if pd.isna(text): return np.nan
    match = re.search(r'(\d+)\s*(?:cell|cells)', str(text), re.IGNORECASE)
    if match:
        return float(match.group(1))
    return np.nan

df['battery_wh'] = df['battery'].apply(extract_battery_wh)
df['battery_cells'] = df['battery'].apply(extract_battery_cells)

# Điền giá trị thiếu cho pin
df['battery_wh'] = df['battery_wh'].fillna(df['battery_wh'].median())
df['battery_cells'] = df['battery_cells'].fillna(df['battery_cells'].median())

#  Xử lý các cột còn lại 
# gpu_brand & gpu_class: One Hot Encoding
gpu_brand_dummies = pd.get_dummies(df['gpu_brand'], prefix='gpu_brand')
gpu_class_dummies = pd.get_dummies(df['gpu_class'], prefix='gpu_class')

# gpu_vram: Đảm bảo là số
df['gpu_vram'] = pd.to_numeric(df['gpu_vram'], errors='coerce').fillna(0)

# Tổng hợp DataFrame cuối cùng 
# Chọn các cột số (bao gồm cả các cột Target Encoding đã có sẵn)
numeric_cols = [
    'gpu_vram', 'gpu_brand_te', 'gpu_class_te',
    'screen_brightness_nits', 'screen_srgb_pct', 'screen_ntsc_pct', 'screen_dcip3_pct', 'screen_anti_glare',
    'battery_wh', 'battery_cells'
]

# Nối tất cả lại với nhau nè
df_final = pd.concat([
    df[numeric_cols],
    screen_res_dummies,
    gpu_brand_dummies,
    gpu_class_dummies
], axis=1)

# Lưu file
df_final.to_csv('data_gpu_final.csv', index=False)
print("Đã tạo file data_gpu_final.csv")

Đã tạo file data_gpu_final.csv


In [9]:
import pandas as pd
import re
import numpy as np

# 1. Đọc dữ liệu
try:
    df = pd.read_csv('data_gpu_subset.csv')
except FileNotFoundError:
    print("Không tìm thấy file data_gpu_subset.csv")
    exit()

# Điền N/A cho các cột text gốc bị trống (theo yêu cầu)
text_cols = ['screen_res', 'screen_tech', 'battery', 'gpu_brand', 'gpu_class']
for col in text_cols:
    if col in df.columns:
        df[col] = df[col].fillna('N/A')

# Hàm phân rã và trích xuất đặc trưng

# Xử lý screen_tech
def extract_screen_tech(text):
    text = str(text).lower()
    
    # Trích xuất độ sáng (nits)
    brightness_match = re.search(r'(\d+)\s*nits', text)
    brightness = float(brightness_match.group(1)) if brightness_match else np.nan
    
    # Trích xuất độ phủ màu sRGB
    srgb_match = re.search(r'(\d+(?:\.\d+)?)\s*%\s*srgb|srgb\s*(\d+(?:\.\d+)?)', text)
    srgb = float(srgb_match.group(1) or srgb_match.group(2)) if srgb_match else np.nan
    
    # Trích xuất độ phủ màu NTSC
    ntsc_match = re.search(r'(\d+(?:\.\d+)?)\s*%\s*ntsc|ntsc\s*(\d+(?:\.\d+)?)', text)
    ntsc = float(ntsc_match.group(1) or ntsc_match.group(2)) if ntsc_match else np.nan
    
    # Trích xuất chống chói (Encoding 1/0 luôn cho cột này)
    anti_glare = 1 if 'chống chói' in text else 0
    
    return pd.Series([brightness, srgb, ntsc, anti_glare])

# Xử lý battery
def extract_battery(text):
    text = str(text).lower()
    # Trích xuất Wh
    wh_match = re.search(r'(\d+)\s*wh', text)
    wh = float(wh_match.group(1)) if wh_match else np.nan
    # Trích xuất Cells
    cell_match = re.search(r'(\d+)\s*cell', text)
    cell = float(cell_match.group(1)) if cell_match else np.nan
    return pd.Series([wh, cell])

# Xử lý làm sạch screen_res trước khi One-Hot
def clean_res(text):
    match = re.search(r'(\d+\s*x\s*\d+)', str(text))
    if match:
        return match.group(1).replace(' ', '')
    return 'N/A'

# Áp dụng trích xuất dữ liệu
df[['screen_brightness_nits', 'screen_srgb_percent', 'screen_ntsc_percent', 'screen_anti_glare']] = df['screen_tech'].apply(extract_screen_tech)
df[['battery_wh', 'battery_cells']] = df['battery'].apply(extract_battery)
df['screen_res_cleaned'] = df['screen_res'].apply(clean_res)

# Xử lý giá trị trống cho các cột số (Imputation) để phù hợp train mô hình
# Sử dụng Median để điền vào các giá trị số bị thiếu
numeric_extracted_cols = ['screen_brightness_nits', 'screen_srgb_percent', 'screen_ntsc_percent', 'battery_wh', 'battery_cells']
existing_numeric_cols = ['gpu_vram', 'gpu_brand_te', 'gpu_class_te']
all_numeric_cols = numeric_extracted_cols + existing_numeric_cols

for col in all_numeric_cols:
    if col in df.columns:
        median_val = df[col].median()
        # Nếu cột toàn NaN thì điền 0
        if pd.isna(median_val):
            median_val = 0
        df[col] = df[col].fillna(median_val)

#  One-Hot Encoding
cols_to_ohe = ['screen_res_cleaned', 'gpu_brand', 'gpu_class']
# Tạo dummies và ghép vào dataframe chính
dummies = pd.get_dummies(df[cols_to_ohe], prefix=cols_to_ohe, dtype=int)
df_final = pd.concat([df, dummies], axis=1)

# Lưu file
output_filename = 'data_gpu_processed.csv'
df_final.to_csv(output_filename, index=False)

print(f"Đã xử lý xong và lưu tại: {output_filename}")
print(f"Các cột mới: {list(dummies.columns) + numeric_extracted_cols}")

Đã xử lý xong và lưu tại: data_gpu_processed.csv
Các cột mới: ['screen_res_cleaned_1080x1920', 'screen_res_cleaned_1200x1920', 'screen_res_cleaned_1366x768', 'screen_res_cleaned_1440x2560', 'screen_res_cleaned_1600x2560', 'screen_res_cleaned_1920x1080', 'screen_res_cleaned_1920x1200', 'screen_res_cleaned_1920x1280', 'screen_res_cleaned_2048x1280', 'screen_res_cleaned_2160x1350', 'screen_res_cleaned_2160x1440', 'screen_res_cleaned_2160x3840', 'screen_res_cleaned_2240x1400', 'screen_res_cleaned_2256x1504', 'screen_res_cleaned_2560x1080', 'screen_res_cleaned_2560x1440', 'screen_res_cleaned_2560x1600', 'screen_res_cleaned_2880x1620', 'screen_res_cleaned_2880x1800', 'screen_res_cleaned_2880x1920', 'screen_res_cleaned_3000x1876', 'screen_res_cleaned_3000x2000', 'screen_res_cleaned_3072x1920', 'screen_res_cleaned_3200x1800', 'screen_res_cleaned_3200x2000', 'screen_res_cleaned_3456x2160', 'screen_res_cleaned_3840x2160', 'screen_res_cleaned_3840x2400', 'screen_res_cleaned_720x1280', 'screen_res

In [10]:
# Verify row counts for original and encoded files
import pandas as pd
import os
print('data_full_final rows:', pd.read_csv('data_full_final.csv').shape[0])
print('data_gpu_encoded_oof rows:', pd.read_csv('data_gpu_encoded_oof.csv').shape[0])
print('data_optimized_encoding', pd.read_csv('data_optimized_encoding.csv').shape[0])
print('data_gpu_processed exists:', os.path.exists('data_gpu_processed.csv'))
if os.path.exists('data_gpu_processed.csv'):
    print('data_gpu_processed rows:', pd.read_csv('data_gpu_processed.csv').shape[0])

data_full_final rows: 6372
data_gpu_encoded_oof rows: 6339
data_optimized_encoding 5298
data_gpu_processed exists: True
data_gpu_processed rows: 6339
